<a href="https://colab.research.google.com/github/130ndim/dul_2021/blob/hw9/Homework/hw9/Homework9_ebm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!if [ -d dul_2021 ]; then rm -Rf dul_2021; fi
!git clone https://github.com/GrigoryBartosh/dul_2021
!pip install ./dul_2021

Cloning into 'dul_2021'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 303 (delta 80), reused 70 (delta 56), pack-reused 163
Receiving objects: 100% (303/303), 54.17 MiB | 41.42 MiB/s, done.
Resolving deltas: 100% (137/137), done.
Processing ./dul_2021
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for dul-2021: filename=dul_2021-0.1.0-py3-none-any.whl size=24553 sha256=6787f0aa059821754451c26535dc83090162ba2af4afdf2c2ac692abd9df46e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-onh0n5ly/wheels/55/59/29/0fb1c6356521577

In [3]:
from dul_2021.utils.hw9_utils import *

# Question 1: EBM on MNIST

Here we will implement EBM on MNIST data. 

* **Architecture.** You may experiment with different hyperparameters and architecture designs, but 
the following designs for the EBM architecture may be useful.

```
    Conv2d(1, 16, 5, 2, 4), 
    Swish(),
    Conv2d(16, 32, 3, 2, 1),
    Swish(), 
    Conv2d(32, 64, 3, 2, 1), 
    Swish(), 
    Conv2d(64, 64, 3, 2, 1), 
    Swish(),
    Flatten(),
    Linear(256, 64), 
    Swish(),
    Linear(64, 1)
```


Where `swish(x) = σ(x)x`. You **should** use activation with non-sparse gradients for better MCMC convergence.

* **Buffer.** To use the contrastive divergence objective, we need to generate samples during training. Because of high dimension of image space we need a lot of iterations in MCMC to obtain reasonable samples. We can reduce sampling cost with buffer trick. The idea of this trick it to re-use previous samples as starting points for MCMC (since distribuionts are close). In our implementation, we initialize 95% of starting points with previous samples and 5% with random noise from -1 to 1. 

* **MCMC.** During MCMC procedure you only need gradients wrt input. You can find it helpful to disable gradients of your model during MCMC. Since all images are in [-1, 1] you should clamp all resulting points during MCMC step. We also clamp gradients to [-0.03, 0.03].
    ```
    for p in model.parameters():
        p.requires_grad = False
    ```

* **Regularizations.**

 You should add small noise to real data otherwise ebm will focus on black areas of real images won't train.

 Add l2 regularaztion to ebm outputs on real and fake data. Otherwise, the output values will fluctuate in a very large range (because energy is invariant to scalar shift).

* **Hyperparameters**
    * Max buffer size - 8192
    * MCMC step size - 10
    * MCMC # step - 60
    * MCMC noise - N(0, 0.005)
    * Noise to data - N(0, 0.005)
    * l2 reg weight - 0.1
    * batch_size - 128
    * use Adam with lr=1e-3 and betas=(0, 0.999)
    * you can exponential scheduler
    * 20 epochs should be enough (~1 hour on gpu)



**You will provide the following deliverables**


1.   Over the course of training, record the regularization and contrastive losses per batch.
2. 100 samples from your trained EBM

In [139]:
from typing import Dict, List, Optional, Tuple, TYPE_CHECKING

import numpy as np

import torch
from torch.optim import Adam
from torch.nn import Module, Conv2d, SiLU as Swish, Sequential as Seq, Linear, Flatten, functional as F
from torch.utils.data import DataLoader, TensorDataset

from tqdm.auto import tqdm

if TYPE_CHECKING:
    import numpy.typing as npt

In [140]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [141]:
class Buffer:
    def __init__(self, img_shape: tuple, size: int = 8192) -> None:
        self._size = size
        self._elements = torch.rand((size,) + img_shape) * 2 - 1

        self._last = 0
        self._num_samples = 0

    def put(self, img: torch.Tensor) -> None:
        B = img.size(0)
        self._num_samples = min(self._size, self._num_samples + B)

        new_last = self._last + B
        if new_last > self._size:
            s = self._size - self._last
            self._elements[-s:] = img[:s]
            self._elements[:B - s] = img[s:]
        else:
            self._elements[self._last:new_last] = img

        self._last = new_last % self._size

    def get(self, n_elements: int) -> torch.Tensor:
        return self._elements[
            torch.from_numpy(np.random.choice(self._num_samples, n_elements, replace=False))
        ]

    def size(self) -> int:
        return self._num_samples

    @property
    def img_shape(self) -> Tuple[int]:
        return tuple(self._elements.size()[1:])

In [142]:
class EBM(Module):
    def __init__(
            self,
            energy: Module,
            buffer: Buffer,
            reg_coeff: float = 0.1,
            buf_samples_ratio: float = 0.05,
            mcmc_n_steps: int = 60,
            mcmc_step_size: float = 10.0,
            mcmc_noise: float = 5e-3,
            data_noise: float = 5e-3,
            clip_sample: Tuple[float, float] = (-1.0, 1.0)
    ) -> None:
        super().__init__()
        self._energy = energy
        self._buffer = buffer
        self._reg_coeff = reg_coeff
        self._buf_samples_ratio = buf_samples_ratio
        self._mcmc_n_steps = mcmc_n_steps
        self._mcmc_step_size = mcmc_step_size
        self._mcmc_noise = mcmc_noise
        self._data_noise = data_noise
        self._clip_sample = clip_sample

    @property
    def device(self) -> torch.device:
        return next(self._energy.parameters()).device

    def _enable_model_grads(self) -> None:
        for p in self._energy.parameters():
            p.requires_grad = True

    def _disable_model_grads(self) -> None:
        for p in self._energy.parameters():
            p.requires_grad = False

    def E(self, input: torch.Tensor, **kwargs) -> torch.Tensor:
        return self._energy(input)

    def mcmc(self, n: int, n_steps: Optional[int] = None, **kwargs) -> torch.Tensor:
        self._disable_model_grads()

        n_buffer_samples = int(self._buf_samples_ratio * n)

        if self._buffer.size() < n_buffer_samples:
            buffer_samples = \
                torch.rand((n_buffer_samples, *self._buffer.img_shape), device=self.device) * 2 - 1
        else:
            buffer_samples = self._buffer.get(n_buffer_samples).to(self.device)

        samples = torch.cat([
            buffer_samples,
            torch.rand((n - n_buffer_samples, *self._buffer.img_shape), device=self.device) * 2 - 1],
            dim=0,
        ).requires_grad_()

        eps = self._mcmc_step_size
        n_steps = n_steps or self._mcmc_n_steps
        for step in range(n_steps):
            z = torch.randn_like(samples) * self._mcmc_noise
            eps -= self._mcmc_step_size / n_steps
            eps = max(eps, 1e-6)

            g = torch.autograd.grad(self.E(samples, **kwargs).sum(), samples)[0].clamp(-0.03, 0.03)
            samples = (samples + z * (2 * eps) ** 0.5 + eps * g).clamp(*self._clip_sample)

        self._buffer.put(samples.detach().cpu())
        self._enable_model_grads()
        return samples

    def _compute_energies(self, real: torch.Tensor):
        batch = torch.cat(
            [real + torch.randn_like(real) * self._data_noise,
             self.mcmc(real.size(0))],
            dim=0
        )
        r_es, m_es = self.E(batch).chunk(2)
        return r_es, m_es

    def _compute_loss(self, real: torch.Tensor, label: torch.Tensor) -> Dict[str, torch.Tensor]:
        r_es, m_es = self._compute_energies(real)

        c_term = m_es.mean() - r_es.mean()
        r_term = (r_es.pow(2) + m_es.pow(2)).mean()

        loss = c_term + self._reg_coeff * r_term
        return {'loss': loss, 'c_term': c_term, 'r_term': r_term}

    def fit(
            self,
            dl: DataLoader,
            num_epochs: int = 20,
            lr: float = 1e-3,
            betas: Tuple[float, float] = (0.0, 0.999),
    ) -> List[Dict[str, float]]:
        log = []

        opt = Adam(self.parameters(), lr=lr, betas=betas)
        with tqdm(total=num_epochs * len(dl)) as pbar:
            for epoch in range(num_epochs):
                for batch in dl:
                    x, y = batch
                    loss_dict = self._compute_loss(x.to(self.device), y.to(self.device))

                    opt.zero_grad()
                    loss_dict['loss'].backward()
                    opt.step()

                    loss_dict = {k: round(v.item(), 4) for k, v in loss_dict.items()}

                    log.append(loss_dict)

                    pbar.update(1)
                    pbar.set_postfix(loss_dict)

        return log

In [143]:
def q1(train_data):
    """
    train_data: An (n_train, 1, 28, 28) torchvision dataset of MNIST images with values from -1 to 1

    Returns
    - a (# of training iterations, ) numpy array full of contrastive losses on each iteration
    - a (# of training iterations, ) numpy array full of regularization losses on each iteration
    - a (100, 28, 28, 1) numpy array of 100 samples from ebm model
    """
    dl = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True, pin_memory=True)


    ebm = EBM(
        energy=Seq(
            Conv2d(1, 16, 5, 2, 4),
            Swish(),
            Conv2d(16, 32, 3, 2, 1),
            Swish(),
            Conv2d(32, 64, 3, 2, 1),
            Swish(),
            Conv2d(64, 64, 3, 2, 1),
            Swish(),
            Flatten(),
            Linear(256, 64),
            Swish(),
            Linear(64, 1)
        ),
        buffer=Buffer(tuple(train_data[0][0].size())),
    )
    ebm.to(DEVICE)
    log = ebm.fit(dl, num_epochs=10)

    c_losses, r_losses = zip(*((d['c_term'], d['r_term']) for d in log))

    samples = ebm.mcmc(100).detach().cpu().numpy().transpose(0, 2, 3, 1)

    return np.array(c_losses), np.array(r_losses), samples

In [119]:
q1_results(q1)

  0%|          | 0/4680 [00:00<?, ?it/s]

0 9.833333333333334
1 9.666666666666668
2 9.500000000000002
3 9.333333333333336
4 9.16666666666667
5 9.000000000000004
6 8.833333333333337
7 8.666666666666671
8 8.500000000000005
9 8.33333333333334
10 8.166666666666673
11 8.000000000000007
12 7.83333333333334
13 7.666666666666673
14 7.500000000000006
15 7.333333333333339
16 7.166666666666672
17 7.000000000000005
18 6.833333333333338
19 6.666666666666671
20 6.500000000000004
21 6.3333333333333375
22 6.1666666666666705
23 6.0000000000000036
24 5.833333333333337
25 5.66666666666667
26 5.500000000000003
27 5.333333333333336
28 5.166666666666669
29 5.000000000000002
30 4.833333333333335
31 4.666666666666668
32 4.500000000000001
33 4.333333333333334
34 4.166666666666667
35 4.0
36 3.8333333333333335
37 3.666666666666667
38 3.5000000000000004
39 3.333333333333334
40 3.1666666666666674
41 3.000000000000001
42 2.8333333333333344
43 2.666666666666668
44 2.5000000000000013
45 2.333333333333335
46 2.1666666666666683
47 2.0000000000000018
48 1.83333

KeyboardInterrupt: 

# Question 2: EBM conditional generation

Here we will train EBM jointly with classifier to generate objects conditioned on class labels. ([original paper](https://arxiv.org/pdf/1912.03263.pdf)).

* **Theory**. Here we will work with joint distribution on x (data point) and y (label). We will parametrize probability of the pair as follows

```
p_θ(x, y) = exp(f_θ(x)[y]) / Z(θ)
```
Where `f(x)[y]` is an y-th output of the classifier model and `Z(θ)` is a normalization constant.

We will optimize log-likelihood of train pair:

```
log p(x, y) = log p(x) + log p(y|x)
```

Second term corresponds to normal classification loss and we will optimize it with cross entropy. We will optimize first tirm via contrastive divergence. The energy of can be from classifire output : `E(x)=-log ∑ exp(f_θ(x)[y])`.

* **Architecture.** We will work with smile dataset and you can use architecture from practice.

* **Hyperparameters**
    * Max buffer size - 8192
    * MCMC step size - 0.1
    * MCMC # step - 500 (during final sampling you can increase it)
    * MCMC noise - N(0, 0.005)
    * Noise to data - None
    * l2 reg weight - 0.1
    * batch_size - 128
    * use Adam with lr=1e-3 and betas=(0, 0.999)
    * you can use exponential scheduler
    * 100 epochs should be enough (~10 minutes on cpu)



**You will provide the following deliverables**


1.   Over the course of training, record loss per batch.
3. 300 samples from your trained EBM. First 100 samples should be generated for label 0, next 100 for label 1, and last 100 for label 2

In [145]:
class CondEBM(EBM):
    def E(self, input: torch.Tensor, label: Optional[npt.ArrayLike] = None, **kwargs) -> torch.Tensor:
        out = self._energy(input)
        if label is not None:
            return out[..., label]
        return out.logsumexp(-1)

    def _compute_loss(self, real: torch.Tensor, label: torch.Tensor) -> Dict[str, torch.Tensor]:
        loss_dict = super()._compute_loss(real, label)
        clf_loss = F.cross_entropy(self._energy(real), label)
        loss_dict['loss'] += clf_loss
        loss_dict['clf_term'] = clf_loss
        return loss_dict

In [146]:
def q2(train_data, train_labels):
    """
    train_data: An (n_train, 2) np.array of binary points
    train_labels: An (n_train, ) np.array labels (3 classes)
    Returns
    - a (# of training iterations, ) numpy array full of losses on each iteration
    - a (300, 2) numpy array of 300 samples from ebm model (0-99 with label 0; 100-199 with label 1; 200-300 with label 2)
    """

    dl = DataLoader(
        TensorDataset(torch.from_numpy(train_data).float(), torch.from_numpy(train_labels)),
        batch_size=128, drop_last=True, shuffle=True
    )

    ebm = CondEBM(
        energy=Seq(
            Linear(2, 256),
            Swish(),
            Linear(256, 256),
            Swish(),
            Linear(256, 3)
        ),
        buffer=Buffer(train_data.shape[1:]),
        mcmc_n_steps=500,
        mcmc_step_size=0.1,
        data_noise=0.0,
        clip_sample=(-3.0, 3.0),
    )
    ebm.to(DEVICE)
    log = ebm.fit(dl, num_epochs=50)
    losses = np.array([d['loss'] for d in log])

    samples = ebm.mcmc(300, n_steps=1000, label=np.array([0] * 100 + [1] * 100 + [2] * 100)).detach().cpu().numpy()
    return losses, samples


In [147]:
q2_results(q2)

  0%|          | 0/150 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Bonus : Conditional generation with EBM and classifier.

Here you need to generate points conditioned on class on smile dataset with independently trained ebm and classifier. For generation use MCMC with

```
log p(x|y) ~ log p(x) + log p(y|x)
```

Where first term is `-E(x)` and second is obtained via backprop from trained classifier.

In [ ]:
def b(train_data, train_labels):
    """
    train_data: An (n_train, 2) np.array of binary points
    train_labels: An (n_train, ) np.array labels (3 classes)
    Returns
    - a (# of training iterations, ) numpy array full of losses of classfier on each iteration
    - a (# of training iterations, ) numpy array full of losses of ebm on each iteration
    - a (300, 2) numpy array of 300 samples from ebm model (0-99 with label 0; 100-199 with label 1; 200-300 with label 2)
    """

In [ ]:
b_results(b)